## Data Download

BAS has an excellent online interface for viewing its available data: 
https://www.bas.ac.uk/project/nagdp/

Be sure to cite Fremand et al. 2022 (https://doi.org/10.5194/essd-14-3379-2022) when talking about releasing data.

This provides a human-usable interface for downloading the data. But, we want to be able to automate it! The portal shows quicklook images corresponding to chunks of lines, though the underlying data files appear to be one-per-flight.


In [1]:
# Once we know the URL of the file, we can grab it with wget

root_dir = "/Users/lindzey/Documents/QIceRadar"
index_dir = "{}/radar_wrangler/data".format(root_dir)
radargram_dir = "{}/data_radargrams".format(root_dir)

institution = "BAS"
survey = "AGAP"
dest_dir = "{}/{}/{}".format(radargram_dir, institution, survey)

url = "https://ramadda.data.bas.ac.uk:443/repository/entry/get/AGAP_A19.nc?entryid=synth%3Aa1abf071-85fc-4118-ad37-7f186b72c847%3AL25ldGNkZi9BR0FQX0ExOS5uYw%3D%3D"
dest_filename = "AGAP_A19.nc"
dest_filepath = "{}/{}".format(dest_dir, dest_filename)
wget_cmd = ["wget", "--no-clobber", "--quiet", "--output-document", dest_filepath, url]

In [16]:
import subprocess

subprocess.check_call(wget_cmd)

0

In order to get the mapping from filename -> download link:
* Go to the landing page. e.g. for AGAP: https://ramadda.data.bas.ac.uk/repository/entry/show?entryid=a1abf071-85fc-4118-ad37-7f186b72c847
* click the "Folder" dropdown next to "netcdf"
* Select "All Actions"
* Select "CSV"; download into data/BAS/netcdf_indices; renaming file to [survey].csv
* Relevant columns are "name" and "url"

Their filename convention is `[survey]_[flight].nc`; all of my references to survey name should match that one.

In [2]:
import os

bas_index_dir = "{}/{}".format(index_dir, institution)
surveys = [ff for ff in os.listdir("{}/{}".format(index_dir, institution)) if ff.endswith(".csv")]
surveys.sort()
print(surveys)

import csv

for ff in surveys:
    survey = ff.split('.')[0]
    filepath = "{}/{}".format(bas_index_dir, ff)
    with open(filepath) as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for flight in csv_reader:
            print("{} {}: {}".format(survey, flight['name'], flight['url']))
            # TODO: Need a lookup from survey to per-survey metadata (e.g. citation)
            
            # TODO: Write download script that assumes data hasn't changed, only re-downloading files that don't exist locally.
            #    Set up ODroid with external drive and have that run over the weekend.
    
            # TODO: Write traj-extractor that saves a (probably subsampled) lat/lon/elev/time to a file
        
            # TODO: Figure out how to create QGIS layer from extracted trajectory.
            #    It should be sorted by institution/survey/{flight|transect} (however the institution chooses to chunk data)
            #    It should include a DOI in the metadata (any way to not repeat this so much, since it'll be one DOI per servey, at least for BAS?)
                     

['AGAP.csv', 'BBAS.csv', 'FISS2015.csv', 'FISS2016.csv', 'GRADES_IMAGE.csv', 'ICEGRAV.csv', 'IMAFI.csv', 'ITGC_2019.csv', 'POLARGAP.csv', 'WISE_ISODYN.csv']
AGAP AGAP_A69E.nc: https://ramadda.data.bas.ac.uk:443/repository/entry/get/AGAP_A69E.nc?entryid=synth%3Aa1abf071-85fc-4118-ad37-7f186b72c847%3AL25ldGNkZi9BR0FQX0E2OUUubmM%3D
AGAP AGAP_A68B.nc: https://ramadda.data.bas.ac.uk:443/repository/entry/get/AGAP_A68B.nc?entryid=synth%3Aa1abf071-85fc-4118-ad37-7f186b72c847%3AL25ldGNkZi9BR0FQX0E2OEIubmM%3D
AGAP AGAP_A68.nc: https://ramadda.data.bas.ac.uk:443/repository/entry/get/AGAP_A68.nc?entryid=synth%3Aa1abf071-85fc-4118-ad37-7f186b72c847%3AL25ldGNkZi9BR0FQX0E2OC5uYw%3D%3D
AGAP AGAP_A67.nc: https://ramadda.data.bas.ac.uk:443/repository/entry/get/AGAP_A67.nc?entryid=synth%3Aa1abf071-85fc-4118-ad37-7f186b72c847%3AL25ldGNkZi9BR0FQX0E2Ny5uYw%3D%3D
AGAP AGAP_A66.nc: https://ramadda.data.bas.ac.uk:443/repository/entry/get/AGAP_A66.nc?entryid=synth%3Aa1abf071-85fc-4118-ad37-7f186b72c847%3AL25ldG

## Data Display

* view radargram in map-view
* plot radargram in 2d
* map from trace to lat/lon
* show surface/bed picks


They also have documentation for how to interact with the netCDF files:
https://antarctica.github.io/PDC_GeophysicsBook/data-portal/Checking_Airborne_Radar_Echo_sounding_data.html

## Questions for Alice

### What is their naming scheme? (suffixes for flight numbers...)
### Is there a list of all surveys? I've found the various rammada links by clicking through on their map interface...
### Will it cause them problems if my radar downloader directly uses the rammada link for fetching data?
### Do they want additional metadata attached to the data in the index layer?

I think a minimum set of metadata would be the data citation:

Citation: Corr, H., Ferraccioli, F., Jordan, T., & Robinson, C. (2021). Processed airborne radio-echo sounding data from the AGAP survey covering Antarctica's Gamburtsev Province, East Antarctica (2007/2009) (Version 1.0) [Data set]. NERC EDS UK Polar Data Centre. https://doi.org/10.5285/A1ABF071-85FC-4118-AD37-7F186B72C847

Replacing [Data set] with flight.

Is there anything else that y'all would like to be included?